In [1]:
import warnings
import pandas as pd
import numpy as np
import os
warnings.filterwarnings('ignore')

In [44]:
name = os.listdir('competition_data')

id_name = []
for i in name:
    temp = i.split(".")[0]
    id_name.append(temp)

In [27]:
# from ckiptagger import data_utils
# data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: C:\Users\WangHongWen\Desktop\data_mining\final_project\ai cup\data.zip
100%|█████████████████████████████████████████████████████████████████████████████| 1.88G/1.88G [12:42<00:00, 2.46MB/s]


# Punctuations

In [29]:
all_data = []
all_data2 = []
from zhon.hanzi import punctuation

for info in os.listdir(r'competition_data'):
#     print(info) # form: xxx.txt
    domain = os.path.abspath(r'competition_data') #獲取資料夾的路徑
    info = os.path.join(domain,info)
    info = open(info,'r', encoding="utf-8")
    temp = info.read()
    temp = temp.replace("\n", "")
    all_data.append(temp)
    for i in punctuation:
        temp = temp.replace(i, '')
    all_data2.append(temp)
    info.close()
# all_data -> with punctuations
# all_data2  -> without punctuations

In [233]:
all_data_c = []
for para in all_data2:
    res = para.replace("\x7f", "")
    all_data_c.append(res)

# Synonyms

In [131]:
chem_syn = pd.read_excel("other_info/Keywords/02chem.list.xlsx")
crop_syn = pd.read_excel("other_info/Keywords/02crop.list.xlsx")
pest_syn = pd.read_excel("other_info/Keywords/02pest.list.xlsx")

chem_syn = chem_syn.fillna(0)
crop_syn = crop_syn.fillna(0)
pest_syn = pest_syn.fillna(0)

In [199]:
def replace_synonyms(para):
    """
    replace all the synonym by the longest one.
    """
    # chem
    s_len = chem_syn.shape[0]
    chem_cols = chem_syn.columns
    for i in range(s_len):
        # find the longest syn.
        base_word = chem_syn['synonym1'][i]
        for c in chem_cols:
            if chem_syn[c][i]!= 0 and len(chem_syn[c][i]) > len(base_word):
                base_word = chem_syn[c][i]
        # replace all syn.
        for col in chem_cols:
            if chem_syn[col][i] != 0:
                para = para.replace(chem_syn[col][i], base_word)
                
    # crop
    s_len = crop_syn.shape[0]
    crop_cols = crop_syn.columns
    for i in range(s_len):
        # find the longest syn.
        base_word = crop_syn['synonym1'][i]
        for c in crop_cols:
            if crop_syn[c][i]!= 0 and len(crop_syn[c][i]) > len(base_word):
                base_word = crop_syn[c][i]
        # replace all syn.
        for col in crop_cols:
            if crop_syn[col][i] != 0:
                para = para.replace(crop_syn[col][i], base_word)

    # pest
    s_len = pest_syn.shape[0]
    pest_cols = pest_syn.columns
    for i in range(s_len):
        # find the longest syn.
        base_word = pest_syn['synonym1'][i]
        for c in pest_cols:
            if pest_syn[c][i]!= 0 and len(pest_syn[c][i]) > len(base_word):
                base_word = pest_syn[c][i]
        # replace all syn.
        for col in pest_cols:
            if pest_syn[col][i] != 0:
                para = para.replace(pest_syn[col][i], base_word)

    return para

In [223]:
r_all_data = [] # replace all synonyms with the first words

for para in all_data2:
    res = replace_synonyms(para).replace("\x7f", "")
    r_all_data.append(res)

# Tokenize

In [32]:
from ckiptagger import WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

C:\anaconda\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\anaconda\lib\site-packages\keras\engine\base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
Exception ignored in: <bound method WS.__del__ of <ckiptagger.api.WS object at 0x0000027E798C5320>>
Traceback (most recent call last):
  File "C:\anaconda\lib\site-packages\ckiptagger\api.py", line 65, in __del__
    self.model.sess.close()
AttributeError: 'WS' object has no attribute 'model'
Exception ignored in: <bound method WS.__del__ of <ckiptagger.api.WS object at 0x0000027E6C007160>>
Tr

In [225]:
word_spilt = ws(r_all_data)

In [235]:
word_spilt2 = ws(all_data_c)

# Calculate the number of tokens

In [259]:
train_lebel = pd.read_csv("other_info/TrainLabel.csv")

index: `id_name`

raw_text: `all_data_c`

tokens_rep: `word_spilt`

tokens: `word_spilt2`

In [270]:
# cols = ['index', 'raw_text', 'tokens_rep', 'tokens']
processed_data = pd.DataFrame({'doc_index': id_name, 'raw_text': all_data_c, 'tokens_rep': word_spilt, 'tokens':word_spilt2})
processed_data['tokens_num'] = ""
processed_data['reference'] = ""

In [271]:
p_len = len(processed_data['doc_index'])
for i in range(p_len):
    processed_data['tokens_num'][i] = len(processed_data['tokens_rep'][i])
    idx = processed_data['doc_index'][i]
    processed_data['reference'][i] = list(train_lebel.query('Test == @idx')['Reference'])

In [272]:
processed_data.to_csv("processed_data.csv", index=False)

In [3]:
df = pd.read_csv("processed_data.csv")
df.describe()

,doc_index,tokens_num
count,560.000000,560.000000
mean,690.512500,339.558929
std,407.902352,148.236648
min,1.000000,18.000000
25%,325.750000,236.750000
50%,691.500000,313.000000
75%,1037.500000,409.500000
max,1401.000000,1087.000000


In [5]:
df[:10]

,doc_index,raw_text,tokens_rep,tokens,tokens_num,reference
0,1,梅雨季來臨文旦黑點病易發生請注意病徵以及早加強防治措施5月已進入梅雨季節近日連續降雨為文旦黑...,"['梅雨季', '來臨', '麻豆', '文旦', '黑點病', '易', '發生', '請...","['梅雨季', '來臨', '文旦', '黑點病', '易', '發生', '請', '注意...",159,[]
1,10,天氣多變溫差大近山區及偏施氮肥田區稻熱病發病較為嚴重籲請農友注意防治花蓮區農改場防檢局及田邊...,"['天氣', '多變', '溫差', '大近', '山區', '及', '偏施', '氮肥田...","['天氣', '多變', '溫差', '大近', '山區', '及', '偏施', '氮肥田...",302,[]
2,1000,新聞稿-稻熱病進入好發季節防檢局籲請農友加強防範行政院農業委員會動植物防疫檢疫局以下簡稱防檢...,"['新聞稿', '-', '稻熱病', '進入', '好', '發', '季節', '防檢局...","['新聞稿', '-', '稻熱病', '進入', '好', '發', '季節', '防檢局...",421,"[1005, 1023]"
3,1005,稻熱病進入好發季節防檢局籲請農友加強防治農委會防檢局表示自3月起全國各地水稻生長陸續進入分蘗...,"['稻熱病', '進入', '好', '發', '季節', '防檢局', '籲請', '農友...","['稻熱病', '進入', '好', '發', '季節', '防檢局', '籲請', '農友...",409,"[1000, 1023]"
4,1007,乍暖還寒防檢局籲請農友加強防治稻熱病農委會防檢局表示全國各地水稻生長陸續進入分蘗期因逢暖冬且...,"['乍暖還寒', '防檢局', '籲請', '農友', '加強', '防治', '稻熱病',...","['乍暖還寒', '防檢局', '籲請', '農友', '加強', '防治', '稻熱病',...",379,"[438, 893, 1000, 1005, 1015, 1023]"
5,1010,提早入梅請加強防範水稻稻熱病發生農委會防檢局表示依據中央氣象局資料今年首波梅雨鋒面提早於五月...,"['提早', '入梅', '請', '加強', '防範', '水稻', '稻熱病', '發生...","['提早', '入梅', '請', '加強', '防範', '水稻', '稻熱病', '發生...",336,"[188, 289, 296, 809, 891, 1000, 1005, 1023]"
6,1011,果實蠅已進入發生高峰期請果農加強防治農委會防檢局表示隨著氣溫日漸攀升夏季水果陸續進入成熟期各...,"['果實蠅', '已', '進入', '發生', '高峰', '期請', '果農', '加強...","['果實蠅', '已', '進入', '發生', '高峰', '期請', '果農', '加強...",376,[]
7,1015,稻熱病蠢蠢欲動防檢局籲請農友加強防範目前水稻生長陸續進入分蘗期由於最近氣候乍暖還寒溫度變化大...,"['稻熱病', '蠢蠢欲動', '防檢局', '籲請', '農友', '加強', '防範',...","['稻熱病', '蠢蠢欲動', '防檢局', '籲請', '農友', '加強', '防範',...",446,"[438, 893, 1000, 1005, 1007, 1023]"
8,1016,莫拉克颱風災後農作物復育措施莫拉克颱風造成的淹水情況使農田環境濕度提高加上天氣放晴後溫度升高...,"['莫拉克', '颱風', '災', '後', '農作物', '復育', '措施', '莫拉...","['莫拉克', '颱風', '災', '後', '農作物', '復育', '措施', '莫拉...",296,[]
9,1023,嘉義市已發現水稻稻熱病發生籲請農友掌握防治時機嘉義市政府防檢局及田邊好幫手關心您嘉義市地區水...,"['嘉義市', '已', '發現', '水稻', '稻熱病', '發生', '籲請', '農...","['嘉義市', '已', '發現', '水稻', '稻熱病', '發生', '籲請', '農...",225,"[1000, 1005]"
